En este notebook se realizó el proceso para los puntos 1 y 2 del proyecto, que consistió en importar las imágenes al ambiente de trabajo, procesarlas de manera adecuada, y crear con ello un primer modelo base

In [2]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Mon Nov 17 06:44:27 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-80GB          Off |   00000000:00:05.0 Off |                    0 |
| N/A   33C    P0             53W /  400W |       0MiB /  81920MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

Con este bloque de código solo se revisa que se esté utilizando GPU para el entrenamiento del modelo.

In [1]:
import cv2 as cv
import numpy as np
import matplotlib as plt
import pandas as pd
import tensorflow as tf

from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical
from google.colab import drive
drive.mount('/content/drive')

from tensorflow.keras.preprocessing.image import ImageDataGenerator


Mounted at /content/drive


En esta seccion del código se importan las liberías y funciones necesarias para la contruccion de CNN

In [3]:
# Verificar que la ruta exista
!ls "/content/drive/MyDrive/Proyecto Final IA/Train"


0  1  2  3  4  5  6  7	8  9


En esta línea de código se verifica que existan las carpetas en Drive con las que se van a trabajar

In [4]:
# =============== PARÁMETROS BÁSICOS ===============
img_height, img_width = 28, 28
batch_size = 32
epochs = 15

# RUTA
train_dir = "/content/drive/MyDrive/Proyecto Final IA/Train"


En este bloque de código se declaran algunos parámetros básicos para el modelo, como lo son las dimensiones de la imágen, así como el tamaño del batch que se utilizará, y las épocas. Además se declara la ruta de donde se seleccionarán las imágenes de entrenamiento.

In [5]:
# =============== GENERADORES (TRAIN / VAL) ===============
datagen = ImageDataGenerator(
    rescale=1./255,       # normaliza a [0,1]
    validation_split=0.2  # 80% train, 20% validación
)

train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    color_mode='grayscale',     # imágenes en escala de grises (28x28x1)
    batch_size=batch_size,
    class_mode='categorical',   # one-hot para 10 clases
    subset='training',
    shuffle=True
)

val_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    color_mode='grayscale',
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation',
    shuffle=False
)

print("Clases detectadas:", train_generator.class_indices)

Found 5126 images belonging to 10 classes.
Found 1277 images belonging to 10 classes.
Clases detectadas: {'0': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9}


En esta sección se utiliza la función ImageDataGenerator para procesar (normalizar y escalar) las imágenes, así como colocarles los *labels* a cada una tanto en el subconjunto de entrenamiento como en el de valización.

In [6]:
model=models.Sequential()
model.add(layers.Conv2D(32,(3,3),padding='same',activation='relu',input_shape=(28,28,1)))

model.add(layers.MaxPooling2D((2,2)))

model.add(layers.Flatten())
# Capa oculta 1
model.add(layers.Dense(128, activation="relu"))

#Capa oculta 2
model.add(layers.Dense(64,activation='relu'))

# Output Layer

model.add(layers.Dense(10,activation='softmax'))

/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


En esta seccion de código se crea la arquitectura del primer modelo. Observamos que este primer modelo tiene 1 capa convolucional con su respectivo MaxPool, Flatten (que perimte pasar de un tensor multidimensional a un tensor de primer orden, es decir a un vector con puros datos), además de 2 capas densas con activación *ReLU*, y una última capa densa con activación *SoftMax* como capa de toma de decisión.

In [7]:
#Optimizador
opt=tf.keras.optimizers.Adam(learning_rate=0.001)

# Funcion de paro
from tensorflow.keras.callbacks import EarlyStopping
early_stop=EarlyStopping(monitor='val_accuracy',patience=10,restore_best_weights=True)

model.compile(optimizer=opt,loss='categorical_crossentropy',metrics=['accuracy'])

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 28, 28, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 14, 14, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 6272)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       802,944 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 10)             │           650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 812,170 (3.10 MB)

 Trainable params: 812,170 (3.10 MB)

 Non-trainable params: 0 (0.00 B)

En esta sección se selecciona el optimizador *Adam* además de definir la función de paro con paciencia de 10. También se despliega un resumen de la arquitectura enteriormente seleccionada. Vemos que la primer capa tiene shape 28 x 28 x 32, y 320 parámetros, que surge de las 32 neuronas declaradas en el bloque anterior, por cada categoría o digito, es decir 32 x 10. En el MaxPooling vemos que el shape cambia a 14 x 14 x 32, por lo que probablemente el "filtro" o ventana utilzada fue de 2 x 2. Después, la capa *Flatten* pasa de un tensor multidimensional, como lo es 14 x 14 x 32, a un vector con puros datos, por lo que ahora el shape resulta de multiplicar 14 x 14 x 32=6272, como se observa en el resumen. Después se declaró una capa densa de 128 neuronas, por lo que los parámetros resultantes se obtienen al hacer (6272 x 128)+128=802,944. El mismo proceso se realiza para obtener el número de parámetros para el resto de las capas.

In [8]:
history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=val_generator,callbacks=[early_stop]
)

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/15
161/161 ━━━━━━━━━━━━━━━━━━━━ 4634s 29s/step - accuracy: 0.2132 - loss: 2.1949 - val_accuracy: 0.5090 - val_loss: 1.6339
Epoch 2/15
161/161 ━━━━━━━━━━━━━━━━━━━━ 49s 303ms/step - accuracy: 0.5732 - loss: 1.3838 - val_accuracy: 0.6327 - val_loss: 1.3304
Epoch 3/15
161/161 ━━━━━━━━━━━━━━━━━━━━ 47s 289ms/step - accuracy: 0.6823 - loss: 1.0269 - val_accuracy: 0.6805 - val_loss: 1.1742
Epoch 4/15
161/161 ━━━━━━━━━━━━━━━━━━━━ 49s 304ms/step - accuracy: 0.7408 - loss: 0.8473 - val_accuracy: 0.7024 - val_loss: 1.1395
Epoch 5/15
161/161 ━━━━━━━━━━━━━━━━━━━━ 50s 310ms/step - accuracy: 0.7788 - loss: 0.7215 - val_accuracy: 0.7024 - val_loss: 1.1131
Epoch 6/15
161/161 ━━━━━━━━━━━━━━━━━━━━ 48s 295ms/step - accuracy: 0.8184 - loss: 0.5925 - val_accuracy: 0.7134 - val_loss: 1.1036
Epoch 7/15
161/161 ━━━━━━━━━━━━━━━━━━━━ 50s 310ms/step - accuracy: 0.8346 - loss: 0.5544 - val_accuracy: 0.7353 - val_loss: 1.0945
Epoch 8/15
161/161 ━━━━━━━━━━━━━━━━━━━━ 46s 286ms/step - accuracy: 0.8512 - loss: 0

En este bloque de código se entrenó al modelo con los datos de entrenamiento, y vemos que para la última época, terminó con un *accuracy* de 0.9372.

In [9]:
val_loss, val_acc = model.evaluate(val_generator)
print("Pérdida en validación:", val_loss)
print("Accuracy en validación:", val_acc)

40/40 ━━━━━━━━━━━━━━━━━━━━ 7s 176ms/step - accuracy: 0.7420 - loss: 1.5080
Pérdida en validación: 1.2392226457595825
Accuracy en validación: 0.7454972863197327


En esta sección, se evaluó el desempeño del modelo con el subconjunto de datos de validación generado mediante *ImageDataGenerator*. Observamos que el *accuracy* de este primer modelo en el subconjunto de datos de validación no fue tan bueno, siendo de 0.7455, y por lo mismo sabemos que hay bastantes áreas de oportunidad.

In [10]:
# Guardar el modelo

model.save("model1.keras")